In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import regexp_extract

In [25]:
from pyspark.sql.functions import col, substring, regexp_replace

In [26]:
from pyspark.sql.types import IntegerType

In [5]:
import pyspark.sql.functions as f

In [6]:
spark=SparkSession.builder.appName('Basics').getOrCreate()

#### Reading Dataset

In [9]:
df=spark.read.csv('SalaryDataset.csv',header=True)

In [10]:
df.show()

+--------------------+--------------+-----------------+---------+-------------+
|        Company Name|     Job Title|Salaries Reported| Location|       Salary|
+--------------------+--------------+-----------------+---------+-------------+
|            Mu Sigma|Data Scientist|              105|Bangalore| ₹6,48,573/yr|
|                 IBM|Data Scientist|               95|Bangalore|₹11,91,950/yr|
|Tata Consultancy ...|Data Scientist|               66|Bangalore| ₹8,36,874/yr|
|    Impact Analytics|Data Scientist|               40|Bangalore| ₹6,69,578/yr|
|           Accenture|Data Scientist|               32|Bangalore| ₹9,44,110/yr|
|             Infosys|Data Scientist|               30|Bangalore| ₹9,08,764/yr|
|           Capgemini|Data Scientist|               28|Bangalore| ₹9,26,124/yr|
|Cognizant Technol...|Data Scientist|               26|Bangalore| ₹7,36,708/yr|
|Anheuser-Busch InBev|Data Scientist|               25|Bangalore|₹16,46,721/yr|
|             Fractal|Data Scientist|   

#### Summarizing and Cleaning Dataset

In [11]:
df.columns

['Company Name', 'Job Title', 'Salaries Reported', 'Location', 'Salary']

In [12]:
df.printSchema()

root
 |-- Company Name: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Salaries Reported: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Salary: string (nullable = true)



In [13]:
df=df.withColumnRenamed('Company Name','Company_Name')

In [14]:
df=df.withColumnRenamed('Job Title','Job_Title')

In [15]:
df=df.withColumnRenamed('Salaries Reported','Salaries_Reported')

In [16]:
df=df.na.drop()

#### Salary Column Cleanup

In [17]:
df=df.withColumn("salary",regexp_replace("salary",",",""))

In [18]:
df=df.withColumn("Currency",regexp_extract("salary",'(\D+)',1))

In [19]:
df=df.withColumn("Amount",regexp_extract("salary",'(\\D+)(\\w+)',2))

In [20]:
df=df.withColumn("Period",substring(df['salary'], -2, 2))

In [21]:
df.show()

+--------------------+--------------+-----------------+---------+-----------+--------+-------+------+
|        Company_Name|     Job_Title|Salaries_Reported| Location|     salary|Currency| Amount|Period|
+--------------------+--------------+-----------------+---------+-----------+--------+-------+------+
|            Mu Sigma|Data Scientist|              105|Bangalore| ₹648573/yr|       ₹| 648573|    yr|
|                 IBM|Data Scientist|               95|Bangalore|₹1191950/yr|       ₹|1191950|    yr|
|Tata Consultancy ...|Data Scientist|               66|Bangalore| ₹836874/yr|       ₹| 836874|    yr|
|    Impact Analytics|Data Scientist|               40|Bangalore| ₹669578/yr|       ₹| 669578|    yr|
|           Accenture|Data Scientist|               32|Bangalore| ₹944110/yr|       ₹| 944110|    yr|
|             Infosys|Data Scientist|               30|Bangalore| ₹908764/yr|       ₹| 908764|    yr|
|           Capgemini|Data Scientist|               28|Bangalore| ₹926124/yr|     

In [22]:
df.describe().show()

+-------+---------------+--------------------+-----------------+---------+----------+--------+-----------------+------+
|summary|   Company_Name|           Job_Title|Salaries_Reported| Location|    salary|Currency|           Amount|Period|
+-------+---------------+--------------------+-----------------+---------+----------+--------+-----------------+------+
|  count|           4339|                4339|             4339|     4339|      4339|    4339|             4339|  4339|
|   mean|           null|                null|2.777137589306292|     null|      null|    null|768070.9638165475|  null|
| stddev|           null|                null|5.148094762075859|     null|      null|    null|769002.4850658625|  null|
|    min|& Other Stories|Associate Machine...|                1|Bangalore|$100488/yr|       $|              100|    hr|
|    max|Águas Guariroba|Software Engineer...|               95|     Pune|₹999998/yr|       ₹|           999998|    yr|
+-------+---------------+---------------

In [27]:
df=df.withColumn("Amount",df["Amount"].cast(IntegerType()))

In [28]:
df=df.withColumn('Salary_INR',f.when(f.col('Currency')=='$',f.col('Amount')*82).when(f.col('Currency')=='£',f.col('Amount')*102).when(f.col('Currency')=='AFN',f.col('Amount')*0.94).otherwise(f.col('Amount')))

In [29]:
df=df.withColumn('Salary_INR',f.when(f.col('Period')=='hr',f.col('Salary_INR')*160).otherwise(f.col('Salary_INR')))

In [30]:
df=df.withColumn('Salary_INR',f.when(f.col('Period')=='mo',f.col('Salary_INR')*12).otherwise(f.col('Salary_INR')))

In [31]:
df.select('Currency').distinct().show()

+--------+
|Currency|
+--------+
|       ₹|
|    AFN |
|       $|
|       £|
+--------+



In [37]:
df.show()

+--------------------+--------------+-----------------+---------+-----------+--------+-------+------+----------+
|        Company_Name|     Job_Title|Salaries_Reported| Location|     salary|Currency| Amount|Period|Salary_INR|
+--------------------+--------------+-----------------+---------+-----------+--------+-------+------+----------+
|            Mu Sigma|Data Scientist|              105|Bangalore| ₹648573/yr|       ₹| 648573|    yr|  648573.0|
|                 IBM|Data Scientist|               95|Bangalore|₹1191950/yr|       ₹|1191950|    yr| 1191950.0|
|Tata Consultancy ...|Data Scientist|               66|Bangalore| ₹836874/yr|       ₹| 836874|    yr|  836874.0|
|    Impact Analytics|Data Scientist|               40|Bangalore| ₹669578/yr|       ₹| 669578|    yr|  669578.0|
|           Accenture|Data Scientist|               32|Bangalore| ₹944110/yr|       ₹| 944110|    yr|  944110.0|
|             Infosys|Data Scientist|               30|Bangalore| ₹908764/yr|       ₹| 908764|  